In [197]:
import numpy as np
from natsort import natsorted
from io import BytesIO
import matplotlib.pyplot as plt
from scipy.optimize import nnls
from IPython.display import clear_output
from numba import jit

plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rc('font', size=12) 
%config InlineBackend.figure_format = 'svg'

# note - if you change settings - beam type, spectrum bin widths, etc, you will need to adjust the plot labels. 
# the value for alphja from the L-curve method, for example, is set by visually looking at that graph. 
# they do not update automatically.

beam_type = 'iso' # use isotropic beam, other options are 'cos' for lambertian or 'beam1d' for pencil beam

# x-ray spectrum discretization. min is 30 keV, max is 1000 keV, min step is 10 keV
x_low_keV =  30   # around 30 keV works well and captures the low energy inflection point;
x_high_keV = 500  # should be left at around 500 keV, X-ray spectra do not usually extend farther
x_bin_keV =  1    # should be left at 1 keV. There is no benefit to coarser binning here and it makes the condition number worse.

# electron spectrum discretization. min is 70 keV, max is 2000 keV, min step is 10 keV
e_low_keV =  50    # suggest 100 keV
e_high_keV = 1000  # should be set to the lowest value you expect from your X-ray data
e_bin_keV =  50    # minimum 10 keV, step size is 10 keV. 

g4_response_data = np.load('/Users/patrick/Documents/phd/data/g4_response.zip')
response = []
file_names = []

# assemble the matrix which maps electron spectra to X-ray spectra by matrix multiplication
for file in natsorted(g4_response_data.files): # arranged by energy
  if (beam_type in file):                      # use the chosen beam type (iso, cos, or beam1d)
    file_names.append(file)
for i in range(int(e_low_keV/10), int((e_high_keV+10)/10), int(e_bin_keV/10)): # energies are sampled in multiples of 10 keV
  response.append(np.loadtxt(BytesIO(g4_response_data[file_names[i]]), dtype='float', delimiter=',', skiprows=9)[x_low_keV:x_high_keV:x_bin_keV,3]/5000000.0)
response = np.array(response).T

# make ranges for X-ray and electron spectrum energies
xspace = np.linspace(x_low_keV,x_high_keV,response.shape[0])
espace = np.linspace(e_low_keV,e_high_keV,response.shape[1])

#  implement tikhonov regularization of different orders with optional non-negativity constraint and preconditioner
def tk(r, alpha, x, order, leftprecon, rightprecon, nneg):
  
  right = []
  for i in range(0, r.shape[1]):
    norm = np.linalg.norm(r[:,i])
    if (norm > 0):
      right.append(1.0/np.linalg.norm(r[:,i]))
    else: 
      right.append(1.0)
  right = np.diag(np.array(right))

  left = []
  for i in range(0, r.shape[0]):
    norm = np.linalg.norm(r[i,:])
    if (norm > 0):
      left.append(1.0/np.linalg.norm(r[i,:]))
    else: 
      left.append(1.0)
  left = np.diag(np.array(left))

  if (not leftprecon):
    left = np.identity(r.shape[0])
  if (not rightprecon):
    right = np.identity(r.shape[1])

  x = np.matmul(left, x)
  R = np.matmul(left,np.matmul(r, right))
  I = np.identity(R.shape[1])          # use for zeroth-order
  L1 = np.copy(I) 
  for i in range(0, R.shape[1] - 1):   # use for first-order
    L1[i,i+1] = -1.0
  L2 = np.copy(I) 
  for i in range(1, R.shape[1]-1):     # use for second-order
    L2[i,i] = -2.0
    L2[i,i+1] = 1.0 
    L2[i,i-1] = 1.0
  L2[0,0] = -2
  L2[0,1] = 1
  L2[-1,-2] = -1
  L2[-1,-1] = -2
  if (order == None):
    op = I*0.0
  elif (order == 0):
    op = I
  elif (order == 1):
    op = L1
  elif (order == 2):
    op = L2
  else:
    return(-1)
  C = np.matmul(np.transpose(R),R) + (alpha**2)*np.matmul(np.transpose(op),op) # normal equations provide explicit solution
  D = np.matmul(np.transpose(R), x)

  if (nneg):
    sol = np.matmul(nnls(C,D,maxiter=10**18)[0],right)      # solve using non-negative least-squares
  else:
    sol = np.matmul(np.linalg.solve(C,D),right)

  error = np.linalg.norm(np.matmul(R,sol) - x)            # difference between the regularized solution and the least-squares solution
  penalty = np.linalg.norm(sol)                           # in zeroth-order tikhonov, the penalty is the norm of the solution
  return(sol,error,penalty)                               # keep track of the error and penalty terms 

# apply leave-one-out cross validation to choose alpha. We choose the alpha that minimizes the error term when x-ray data points are suppressed. 
def cv(r, alphas, x, order, rightprecon, leftprecon, nneg):
    err = []
    for alpha in alphas:
        e = 0
        for i in range(0, len(x)-1,20):                            # this is expensive
            response_truncated = np.delete(response,i,0)       
            x_truncated = np.delete(x,i)                        # delete one X-ray data point and solve the reduced problem
            sol = tk(response_truncated, alpha, x_truncated, order, rightprecon, leftprecon, nneg)[0]
            sol[-1] = 0.0
            e += (np.matmul(response,sol)[i] - x[i])**2 # add the error between the resulting solution and the missing data point
        err.append(e)                                             # this value of alpha gave this total error between fit models and suppressed data points
    return(err) 

In [198]:
# exponential beams across folding factors and intensities

beams = []
beam_signals = []
sols = []

intensities = np.full(25,1e9)
loocv_alphas = np.logspace(2,5,10)
foldings = [100,200,300]
i = 0
for folding in foldings:
    i = 0
    for intensity in intensities:
        clear_output(wait=True)
        print('computing folding: ' + str(folding) + ' run: ' + str(i))
        beam = np.exp(-espace/folding)             # exponential electron beam, folding factor of 250 keV
        beam *= intensity/np.trapz(beam,espace)        # total intensity is 1e10 electrons / cm^2 / sec
        beam_signal = np.matmul(response, beam)    # compute X-ray response from the electron beam
        for e in range(0, len(xspace)):  # add some measurement noise. This is poisson in reality but use a normal distribution
            beam_signal[e] = np.random.normal(beam_signal[e], scale=np.sqrt(beam_signal[e]))
        beams.append(beam)
        beam_signals.append(beam_signal)
        loocv_e = cv(response, loocv_alphas, beam_signal, order=2, rightprecon=True, leftprecon=True, nneg=True)
        sol, error, penalty = tk(response, loocv_alphas[np.argmin(loocv_e)], beam_signal, order=2, rightprecon=True ,leftprecon=True, nneg=True)
        sols.append(sol)
        i = i + 1

computing folding: 200 run: 19


KeyboardInterrupt: 

In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[24]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[24],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Exponential, Folding Factor 100 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[24]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[24],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,300])
ax1.set_ylim([1e-1,500])

plt.savefig('exponential_folding_100keV_posonly_preconditioning_1e9_particles.pdf')

In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[25]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[25:49]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[25],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Exponential, Folding Factor 100 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[25]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[25:49]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[30],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,300])
ax1.set_ylim([1e-1,500])

plt.savefig('exponential_folding_200keV_posonly_preconditioning_1e9_particles.pdf')

In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[50]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[50],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Exponential, Folding Factor 100 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[50]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[50],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,300])
ax1.set_ylim([1e-1,800])

plt.savefig('exponential_folding_300keV_posonly_preconditioning_1e9_particles.pdf')

In [ ]:
# exponential beams across folding factors and intensities, non negativity constraint off

beams = []
beam_signals = []
sols = []

intensities = np.full(25,1e9)
loocv_alphas = np.logspace(2,5,10)
foldings = [50,100,200,300]
i = 0
for folding in foldings:
    i = 0
    for intensity in intensities:
        clear_output(wait=True)
        print('computing folding: ' + str(folding) + ' run: ' + str(i))
        beam = np.exp(-espace/folding)             # exponential electron beam, folding factor of 250 keV
        beam *= intensity/np.trapz(beam,espace)        # total intensity is 1e10 electrons / cm^2 / sec
        beam_signal = np.matmul(response, beam)    # compute X-ray response from the electron beam
        for e in range(0, len(xspace)):  # add some measurement noise. This is poisson in reality but use a normal distribution
            beam_signal[e] = np.random.normal(beam_signal[e], scale=np.sqrt(beam_signal[e]))
        beams.append(beam)
        beam_signals.append(beam_signal)
        loocv_e = cv(response, loocv_alphas, beam_signal, order=2, rightprecon=True, leftprecon=True, nneg=False)
        sol, error, penalty = tk(response, loocv_alphas[np.argmin(loocv_e)], beam_signal, order=2, rightprecon=True ,leftprecon=True, nneg=False)
        sols.append(sol)
        i = i + 1

computing folding: 300 run: 24


In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[25]),'x',color=cmap(0),label='retrieval (n = 25 runs)')
for sol in sols[25:50]:
    ax0.semilogy(espace,(sol),'x',color=cmap(0))
ax0.semilogy(espace, beams[25],linestyle='dashed',color=cmap(1),label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')

ax1.semilogy(xspace,np.matmul(response,sols[25]),color=cmap(0),label='retrieval (n = 25 runs)')
for sol in sols[25:50]:
    ax1.semilogy(xspace,np.matmul(response,sol),color=cmap(0))
ax1.semilogy(xspace, beam_signals[25],color=cmap(1),label='assumed')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')

ax0.set_title('Simulated and Retrieved Electron Spectra\n Exponential, Folding Factor 100 keV\n')

plt.savefig('exponential_folding_100keV_preconditioning.pdf')

In [ ]:
# exponential beams across folding factors and intensities

beams = []
beam_signals = []
sols = []

intensities = np.full(25,1e8)
loocv_alphas = np.logspace(2,5,10)
foldings = [100,200,300]
i = 0
for folding in foldings:
    i = 0
    for intensity in intensities:
        clear_output(wait=True)
        print('computing folding: ' + str(folding) + ' run: ' + str(i))
        beam = np.exp(-espace/folding)             # exponential electron beam, folding factor of 250 keV
        beam *= intensity/np.trapz(beam,espace)        # total intensity is 1e10 electrons / cm^2 / sec
        beam_signal = np.matmul(response, beam)    # compute X-ray response from the electron beam
        for e in range(0, len(xspace)):  # add some measurement noise. This is poisson in reality but use a normal distribution
            beam_signal[e] = np.random.normal(beam_signal[e], scale=np.sqrt(beam_signal[e]))
        beams.append(beam)
        beam_signals.append(beam_signal)
        loocv_e = cv(response, loocv_alphas, beam_signal, order=2, rightprecon=True, leftprecon=True, nneg=True)
        sol, error, penalty = tk(response, loocv_alphas[np.argmin(loocv_e)], beam_signal, order=2, rightprecon=True ,leftprecon=True, nneg=True)
        sols.append(sol)
        i = i + 1

computing folding: 300 run: 24


In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[24]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[24],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Exponential, Folding Factor 100 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[24]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[24],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,200])
ax1.set_ylim([1e-1,200])

plt.savefig('exponential_folding_100keV_posonly_preconditioning_1e8_particles.pdf')

In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[25]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[25:49]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[25],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Exponential, Folding Factor 100 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[30]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[26:49]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[30],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,300])
ax1.set_ylim([1e-1,500])

plt.savefig('exponential_folding_200keV_posonly_preconditioning_1e8_particles.pdf')

In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[50]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[50],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Exponential, Folding Factor 100 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[50]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[50],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,300])
ax1.set_ylim([1e-1,800])

plt.savefig('exponential_folding_300keV_posonly_preconditioning_1e8_particles.pdf')

In [ ]:
# exponential beams across folding factors and intensities

beams = []
beam_signals = []
sols = []

intensities = np.full(25,1e7)
loocv_alphas = np.logspace(2,5,10)
foldings = [100,200,300]
i = 0
for folding in foldings:
    i = 0
    for intensity in intensities:
        clear_output(wait=True)
        print('computing folding: ' + str(folding) + ' run: ' + str(i))
        beam = np.exp(-espace/folding)             # exponential electron beam, folding factor of 250 keV
        beam *= intensity/np.trapz(beam,espace)        # total intensity is 1e10 electrons / cm^2 / sec
        beam_signal = np.matmul(response, beam)    # compute X-ray response from the electron beam
        for e in range(0, len(xspace)):  # add some measurement noise. This is poisson in reality but use a normal distribution
            beam_signal[e] = np.random.normal(beam_signal[e], scale=np.sqrt(beam_signal[e]))
        beams.append(beam)
        beam_signals.append(beam_signal)
        loocv_e = cv(response, loocv_alphas, beam_signal, order=2, rightprecon=True, leftprecon=True, nneg=True)
        sol, error, penalty = tk(response, loocv_alphas[np.argmin(loocv_e)], beam_signal, order=2, rightprecon=True ,leftprecon=True, nneg=True)
        sols.append(sol)
        i = i + 1

computing folding: 300 run: 24


In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[24]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[24],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Exponential, Folding Factor 100 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[24]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[24],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,200])
ax1.set_ylim([1e-3,100])

plt.savefig('exponential_folding_100keV_posonly_preconditioning_1e7_particles.pdf')

In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[25]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[25:49]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[25],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Exponential, Folding Factor 100 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[30]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[26:49]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[30],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,100])
ax1.set_ylim([1e-1,500])

plt.savefig('exponential_folding_200keV_posonly_preconditioning_1e7_particles.pdf')

In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[50]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[50],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Exponential, Folding Factor 100 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[50]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[50],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,300])
ax1.set_ylim([1e-2,300])

plt.savefig('exponential_folding_300keV_posonly_preconditioning_1e7_particles.pdf')

In [ ]:
# exponential beams across folding factors and intensities

beams = []
beam_signals = []
sols = []

intensities = np.full(25,1e9)
loocv_alphas = np.logspace(2,5,10)
energies = [200, 400, 600]

i = 0
for energy in energies:
    i = 0
    for intensity in intensities:
        clear_output(wait=True)
        print('computing energy: ' + str(energy) + ' run: ' + str(i))
        beam = np.full(len(espace), 0.0)
        beam[(np.abs(espace - energy)).argmin()] = intensity
        beam_signal = np.matmul(response, beam)    # compute X-ray response from the electron beam
        for e in range(0, len(xspace)):  # add some measurement noise. This is poisson in reality but use a normal distribution
            beam_signal[e] = np.random.normal(beam_signal[e], scale=np.sqrt(beam_signal[e]))
        beams.append(beam)
        beam_signals.append(beam_signal)
        loocv_e = cv(response, loocv_alphas, beam_signal, order=2, rightprecon=True, leftprecon=True, nneg=True)
        sol, error, penalty = tk(response, loocv_alphas[np.argmin(loocv_e)], beam_signal, order=2, rightprecon=True ,leftprecon=True, nneg=True)
        sols.append(sol)
        i = i + 1

computing energy: 600 run: 24


In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[0]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[0],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Monoenergetic, Energy 200 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[0]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[0],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,300])

plt.savefig('mono_200keV_posonly_preconditioning_1e9_particles.pdf')

In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[35]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[25:49]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[45],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Monoenergetic, Energy 400 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[25]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[25:49]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[25],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,500])

plt.savefig('mono_400keV_posonly_preconditioning_1e9_particles.pdf')

In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[55]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[55],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Monoenergetic, Energy 600 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[55]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[55],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,500])

plt.savefig('mono_600keV_posonly_preconditioning_1e9_particles.pdf')

In [ ]:
beams = []
beam_signals = []
sols = []

intensities = np.full(25,1e8)
loocv_alphas = np.logspace(2,5,10)
energies = [200, 400, 600]

i = 0
for energy in energies:
    i = 0
    for intensity in intensities:
        clear_output(wait=True)
        print('computing energy: ' + str(energy) + ' run: ' + str(i))
        beam = np.full(len(espace), 0.0)
        beam[(np.abs(espace - energy)).argmin()] = intensity
        beam_signal = np.matmul(response, beam)    # compute X-ray response from the electron beam
        for e in range(0, len(xspace)):  # add some measurement noise. This is poisson in reality but use a normal distribution
            beam_signal[e] = np.random.normal(beam_signal[e], scale=np.sqrt(beam_signal[e]))
        beams.append(beam)
        beam_signals.append(beam_signal)
        loocv_e = cv(response, loocv_alphas, beam_signal, order=2, rightprecon=True, leftprecon=True, nneg=True)
        sol, error, penalty = tk(response, loocv_alphas[np.argmin(loocv_e)], beam_signal, order=2, rightprecon=True ,leftprecon=True, nneg=True)
        sols.append(sol)
        i = i + 1

computing energy: 600 run: 24


In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[0]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[0],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Monoenergetic, Energy 200 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[0]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[0],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,300])

plt.savefig('mono_200keV_posonly_preconditioning_1e8_particles.pdf')

In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[35]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[25:49]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[45],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Monoenergetic, Energy 400 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[25]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[25:49]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[25],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,500])

plt.savefig('mono_400keV_posonly_preconditioning_1e8_particles.pdf')

In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[55]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[55],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Monoenergetic, Energy 600 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[55]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[55],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,500])

plt.savefig('mono_600keV_posonly_preconditioning_1e8_particles.pdf')

In [ ]:
beams = []
beam_signals = []
sols = []

intensities = np.full(25,1e7)
loocv_alphas = np.logspace(2,5,10)
energies = [200, 400, 600]

i = 0
for energy in energies:
    i = 0
    for intensity in intensities:
        clear_output(wait=True)
        print('computing energy: ' + str(energy) + ' run: ' + str(i))
        beam = np.full(len(espace), 0.0)
        beam[(np.abs(espace - energy)).argmin()] = intensity
        beam_signal = np.matmul(response, beam)    # compute X-ray response from the electron beam
        for e in range(0, len(xspace)):  # add some measurement noise. This is poisson in reality but use a normal distribution
            beam_signal[e] = np.random.normal(beam_signal[e], scale=np.sqrt(beam_signal[e]))
        beams.append(beam)
        beam_signals.append(beam_signal)
        loocv_e = cv(response, loocv_alphas, beam_signal, order=2, rightprecon=True, leftprecon=True, nneg=True)
        sol, error, penalty = tk(response, loocv_alphas[np.argmin(loocv_e)], beam_signal, order=2, rightprecon=True ,leftprecon=True, nneg=True)
        sols.append(sol)
        i = i + 1

computing energy: 600 run: 24


In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[0]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[0],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Monoenergetic, Energy 200 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[0]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[0],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,300])

plt.savefig('mono_200keV_posonly_preconditioning_1e7_particles.pdf')

In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[35]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[25:49]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[45],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Monoenergetic, Energy 400 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[25]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[25:49]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[25],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,500])

plt.savefig('mono_400keV_posonly_preconditioning_1e7_particles.pdf')

In [ ]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.semilogy(espace,(sols[55]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax0.semilogy(espace,(sol),color='lightgrey')
ax0.semilogy(espace, beams[55],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Monoenergetic, Energy 600 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[55]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[55],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,500])

plt.savefig('mono_600keV_posonly_preconditioning_1e7_particles.pdf')

In [260]:
# gaussian beams across widths

beams = []
beam_signals = []
sols = []

intensities = np.full(25,1e9)
loocv_alphas = np.logspace(2,5,10)
sigmas = [50, 100, 150]

for sigma in sigmas:
    i = 0
    for intensity in intensities:
        clear_output(wait=True)
        print('computing sigma: ' + str(sigma) + ' run: ' + str(i))
        beam = np.exp(-np.power(espace - 400, 2.) / (2 * np.power(sigma, 2.)))
        beam = beam*intensity/np.trapz(beam,espace) 
        beam_signal = np.matmul(response, beam)    # compute X-ray response from the electron beam
        for e in range(0, len(xspace)):  # add some measurement noise. This is poisson in reality but use a normal distribution
            beam_signal[e] = np.random.normal(beam_signal[e], scale=np.sqrt(beam_signal[e]))
        beams.append(beam)
        beam_signals.append(beam_signal)
        loocv_e = cv(response, loocv_alphas, beam_signal, order=2, rightprecon=True, leftprecon=True, nneg=True)
        sol, error, penalty = tk(response, loocv_alphas[np.argmin(loocv_e)], beam_signal, order=2, rightprecon=True ,leftprecon=True, nneg=True)
        sols.append(sol)
        i = i + 1

computing sigma: 150 run: 24


In [261]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.plot(espace,(sols[0]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax0.plot(espace,(sol),color='lightgrey')
ax0.plot(espace, beams[0],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Gaussian, Sigma 50 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])

ax1.semilogy(xspace,np.matmul(response,sols[0]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[0],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim(30,300)

plt.savefig('gauss_50keV_posonly_preconditioning_1e9_particles.pdf')

In [262]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.plot(espace,(sols[35]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[25:49]:
    ax0.plot(espace,(sol),color='lightgrey')
ax0.plot(espace, beams[45],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Gaussian, Sigma 100 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])
ax0.set_ylim([0,1e7])

ax1.semilogy(xspace,np.matmul(response,sols[25]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[25:49]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[25],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,500])

plt.savefig('gauss_100keV_posonly_preconditioning_1e9_particles.pdf')

In [263]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.plot(espace,(sols[55]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax0.plot(espace,(sol),color='lightgrey')
ax0.plot(espace, beams[55],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Gaussian, Sigma 150 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])
ax0.set_ylim([0,1e7])

ax1.semilogy(xspace,np.matmul(response,sols[55]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[55],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,500])

plt.savefig('gauss_150keV_posonly_preconditioning_1e9_particles.pdf')

In [254]:
# gaussian beams across widths

beams = []
beam_signals = []
sols = []

intensities = np.full(25,1e8)
loocv_alphas = np.logspace(2,5,10)
sigmas = [50, 100, 150]

for sigma in sigmas:
    i = 0
    for intensity in intensities:
        clear_output(wait=True)
        print('computing sigma: ' + str(sigma) + ' run: ' + str(i))
        beam = np.exp(-np.power(espace - 400, 2.) / (2 * np.power(sigma, 2.)))
        beam = beam*intensity/np.trapz(beam,espace) 
        beam_signal = np.matmul(response, beam)    # compute X-ray response from the electron beam
        for e in range(0, len(xspace)):  # add some measurement noise. This is poisson in reality but use a normal distribution
            beam_signal[e] = np.random.normal(beam_signal[e], scale=np.sqrt(beam_signal[e]))
        beams.append(beam)
        beam_signals.append(beam_signal)
        loocv_e = cv(response, loocv_alphas, beam_signal, order=2, rightprecon=True, leftprecon=True, nneg=True)
        sol, error, penalty = tk(response, loocv_alphas[np.argmin(loocv_e)], beam_signal, order=2, rightprecon=True ,leftprecon=True, nneg=True)
        sols.append(sol)
        i = i + 1

computing sigma: 150 run: 24


In [256]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.plot(espace,(sols[0]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax0.plot(espace,(sol),color='lightgrey')
ax0.plot(espace, beams[0],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Gaussian, Sigma 50 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])
ax0.set_ylim([0,1e6])

ax1.semilogy(xspace,np.matmul(response,sols[0]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[0],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim(30,300)

plt.savefig('gauss_50keV_posonly_preconditioning_1e8_particles.pdf')

In [257]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.plot(espace,(sols[35]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[25:49]:
    ax0.plot(espace,(sol),color='lightgrey')
ax0.plot(espace, beams[45],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Gaussian, Sigma 100 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])
ax0.set_ylim([0,1e6])

ax1.semilogy(xspace,np.matmul(response,sols[25]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[25:49]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[25],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,500])

plt.savefig('gauss_100keV_posonly_preconditioning_1e8_particles.pdf')

In [259]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.plot(espace,(sols[55]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax0.plot(espace,(sol),color='lightgrey')
ax0.plot(espace, beams[55],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Gaussian, Sigma 150 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])
ax0.set_ylim([0,1e6])

ax1.semilogy(xspace,np.matmul(response,sols[55]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[55],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,500])

plt.savefig('gauss_150keV_posonly_preconditioning_1e8_particles.pdf')

In [264]:
# gaussian beams across widths

beams = []
beam_signals = []
sols = []

intensities = np.full(25,1e7)
loocv_alphas = np.logspace(2,5,10)
sigmas = [50, 100, 150]

for sigma in sigmas:
    i = 0
    for intensity in intensities:
        clear_output(wait=True)
        print('computing sigma: ' + str(sigma) + ' run: ' + str(i))
        beam = np.exp(-np.power(espace - 400, 2.) / (2 * np.power(sigma, 2.)))
        beam = beam*intensity/np.trapz(beam,espace) 
        beam_signal = np.matmul(response, beam)    # compute X-ray response from the electron beam
        for e in range(0, len(xspace)):  # add some measurement noise. This is poisson in reality but use a normal distribution
            beam_signal[e] = np.random.normal(beam_signal[e], scale=np.sqrt(beam_signal[e]))
        beams.append(beam)
        beam_signals.append(beam_signal)
        loocv_e = cv(response, loocv_alphas, beam_signal, order=2, rightprecon=True, leftprecon=True, nneg=True)
        sol, error, penalty = tk(response, loocv_alphas[np.argmin(loocv_e)], beam_signal, order=2, rightprecon=True ,leftprecon=True, nneg=True)
        sols.append(sol)
        i = i + 1

computing sigma: 150 run: 24


In [265]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.plot(espace,(sols[0]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax0.plot(espace,(sol),color='lightgrey')
ax0.plot(espace, beams[0],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Gaussian, Sigma 50 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])
ax0.set_ylim([0,1e5])

ax1.semilogy(xspace,np.matmul(response,sols[0]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[0:24]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[0],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim(30,300)

plt.savefig('gauss_50keV_posonly_preconditioning_1e7_particles.pdf')

In [266]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.plot(espace,(sols[35]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[25:49]:
    ax0.plot(espace,(sol),color='lightgrey')
ax0.plot(espace, beams[45],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Gaussian, Sigma 100 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])
ax0.set_ylim([0,1e5])

ax1.semilogy(xspace,np.matmul(response,sols[25]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[25:49]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[25],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,500])

plt.savefig('gauss_100keV_posonly_preconditioning_1e7_particles.pdf')

In [267]:
fig = plt.figure(figsize=(6.52437527778,7.5))
gs = plt.GridSpec(nrows=2, ncols=1,hspace=.5,wspace=.25)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
cmap = plt.get_cmap("tab10")

ax0.plot(espace,(sols[55]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax0.plot(espace,(sol),color='lightgrey')
ax0.plot(espace, beams[55],linestyle='dashed',color='black',label='assumed')
ax0.legend()
ax0.set_xlabel('Electron Energy (keV)')
ax0.set_ylabel('electrons / keV')
ax0.set_title('Simulated and Retrieved Electron Spectra\n Gaussian, Sigma 150 keV\nPositivity Constraint, Preconditioning Applied\n')
ax0.set_xlim([30,800])
ax0.set_ylim([0,1e5])

ax1.semilogy(xspace,np.matmul(response,sols[55]),color='lightgrey',label='retrieval (n = 25 runs)')
for sol in sols[50:-1]:
    ax1.semilogy(xspace,np.matmul(response,sol),color='lightgrey')
ax1.semilogy(xspace, beam_signals[55],color='black',label='assumed',linestyle='solid')
ax1.legend()
ax1.set_xlabel('X-ray Energy (keV)')
ax1.set_ylabel('counts / keV')
ax1.set_xlim([30,500])

plt.savefig('gauss_150keV_posonly_preconditioning_1e7_particles.pdf')